In [8]:
import ollama
import pandas as pd
models = ollama.list()['models'][1:]
models
local_llms = {}
for m in models:
    local_llms[m['model']] = [m['details']['parameter_size'], m['details']['quantization_level'], m['modified_at']]
local_llms

{'gemma:latest': ['9B', 'Q4_0', '2024-02-27T15:17:13.268144+01:00'],
 'llama2:13b': ['13B', 'Q4_0', '2024-02-19T20:25:04.5746316+01:00'],
 'llama2:latest': ['7B', 'Q4_0', '2024-02-19T18:08:56.2153244+01:00'],
 'mistral:latest': ['7B', 'Q4_0', '2024-02-21T11:00:44.2236075+01:00'],
 'mistral-openorca:7b-q8_0': ['7B',
  'Q8_0',
  '2024-02-27T15:33:59.0029113+01:00'],
 'mistral-openorca:latest': ['7B',
  'Q4_0',
  '2024-02-19T21:26:13.5663377+01:00'],
 'phi:latest': ['3B', 'Q4_0', '2024-02-27T15:47:35.4991288+01:00'],
 'solar:latest': ['11B', 'Q4_0', '2024-02-19T21:32:41.2370413+01:00'],
 'vicuna:latest': ['7B', 'Q4_0', '2024-02-19T21:29:23.7879334+01:00'],
 'wizard-vicuna-uncensored:13b': ['13B',
  'Q4_0',
  '2024-02-27T15:45:08.6308363+01:00']}

In [9]:
system_prompts = [
    "You are a senior programmer. I'm gonna tell you about a problem I am trying to solve, and you will provide me a working, clean and well-annotated code solving my problem."
                 ]

In [10]:
otazky = [
    
    f"""I want you to write a Python script that: 1/ loads all JSON files in the current working directory, 2/ joins their content into a pandas dataframe, 3/ groups the dataframe by "model" column, 4/ for each group, calculates the average value in "cas" column, 5/ sorts the resulting series."""

]

In [11]:
odpovedi = []
for o in otazky:
    print(o.upper())
    for m, val in local_llms.items():
        if m != 'gemma:latest':
            for s in system_prompts:
                print(f"\n{m.upper()} | system prompt: {s}\n")
                odpoved = {}
                if s != None:
                    resp = ollama.generate(model=m, prompt=o, system=s)
                else:
                    resp = ollama.generate(model=m, prompt=o)
                odpoved['model'] = m
                odpoved['systemovy_prompt'] = s
                odpoved['prompt'] = o
                odpoved['odpoved'] = resp['response']
                odpoved['cas'] = resp['total_duration']
                odpoved['params'] = val[0]
                odpoved['quant'] = val[1]
                odpoved['verze'] = val[2]
                print(odpoved['odpoved'])
                odpovedi.append(odpoved)

I WANT YOU TO WRITE A PYTHON SCRIPT THAT: 1/ LOADS ALL JSON FILES IN THE CURRENT WORKING DIRECTORY, 2/ JOINS THEIR CONTENT INTO A PANDAS DATAFRAME, 3/ GROUPS THE DATAFRAME BY "MODEL" COLUMN, 4/ FOR EACH GROUP, CALCULATES THE AVERAGE VALUE IN "CAS" COLUMN, 5/ SORTS THE RESULTING SERIES.

LLAMA2:13B | system prompt: You are a senior programmer. I'm gonna tell you about a problem I am trying to solve, and you will provide me a working, clean and well-annotated code solving my problem.


Sure! Here is a Python script that should accomplish what you're looking for:
```
import json
import pandas as pd

# Load all JSON files in the current working directory
files = [f for f in os.listdir() if f.endswith('.json')]

# Create a list of dictionaries, one for each JSON file
data_list = []
for file in files:
    with open(file) as json_file:
        data = json.load(json_file)
        data_list.append(data)

# Convert the list of dictionaries to a pandas dataframe
df = pd.concat(data_list).reset_in

 import pandas as pd
import os
import json


# 1. Load all JSON files in the current working directory
json_files = [f for f in os.listdir() if f.endswith(".json")]
dataframes = []
for file_path in json_files:
    with open(file_path) as f:
        dataframe = pd.read_json(f)
    dataframes.append(dataframe)

# 2. Join all dataframes into one dataframe
full_df = pd.concat(dataframes, ignore_index=True)

# 3. Group the dataframe by "model" column
grouped_df = full_df.groupby("model")

# 4. For each group, calculate the average value in "cas" column
average_values = grouped_df["cas"].mean()

# 5. Sort the resulting series
sorted_values = average_values.sort_index()

print(sorted_values)

This script will output a pandas series with the following content:
```python
model-1: 1.5
model-2: 2.0
model-3: 1.8
```

Note that this is just one possible solution, and there may be other ways to achieve the same result depending on your specific requirements and data structure.


Consider a hypotheti

In [12]:
frejm = pd.DataFrame(odpovedi)
frejm

,model,systemovy_prompt,prompt,odpoved,cas,params,quant,verze
0,llama2:13b,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,\nSure! Here is a Python script that should ac...,747136630600,13B,Q4_0,2024-02-19T20:25:04.5746316+01:00
1,llama2:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,\nSure! I'd be happy to help you with that. Ca...,38443993900,7B,Q4_0,2024-02-19T18:08:56.2153244+01:00
2,mistral:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,Here's the Python script you requested using ...,124451228800,7B,Q4_0,2024-02-21T11:00:44.2236075+01:00
3,mistral-openorca:7b-q8_0,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,Here's a Python script that solves your probl...,197567976400,7B,Q8_0,2024-02-27T15:33:59.0029113+01:00
4,mistral-openorca:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,# Python script to load all JSON files in the...,206943563100,7B,Q4_0,2024-02-19T21:26:13.5663377+01:00
5,phi:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,import pandas as pd\nimport os\nimport json\n...,66223047800,3B,Q4_0,2024-02-27T15:47:35.4991288+01:00
6,solar:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,```python\n # Step 1: Import necessary lib...,213716848300,11B,Q4_0,2024-02-19T21:32:41.2370413+01:00
7,vicuna:latest,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,```python\nimport json\nimport os\nimport pand...,68404293400,7B,Q4_0,2024-02-19T21:29:23.7879334+01:00
8,wizard-vicuna-uncensored:13b,You are a senior programmer. I'm gonna tell yo...,I want you to write a Python script that: 1/ l...,Here's the Python script that meets your requi...,212966304000,13B,Q4_0,2024-02-27T15:45:08.6308363+01:00


In [13]:
frejm.to_json("009_pandas.json")

## Fungují?

### LLAMA2:13B

In [15]:
import json
import pandas as pd

# Load all JSON files in the current working directory
files = [f for f in os.listdir() if f.endswith('.json')]

# Create a list of dictionaries, one for each JSON file
data_list = []
for file in files:
    with open(file) as json_file:
        data = json.load(json_file)
        data_list.append(data)

# Convert the list of dictionaries to a pandas dataframe
df = pd.concat(data_list).reset_index()

# Group the dataframe by "model" column
groups = df.groupby('model')

# For each group, calculate the average value in "cas" column
for group, subdf in groups:
    avg_cas = subdf['cas'].mean()
    print(f'Model {group}: {avg_cas}')

# Sort the resulting series
sorted_series = pd.Series(groups.values(), index=groups.index)
sorted_series.sort_values(by='cas', ascending=False, inplace=True)

NameError: name 'os' is not defined

### LLAMA2:LATEST 

In [16]:
import os
import json
import pandas as pd

# Step 1: Load all JSON files in the current working directory
json_files = [f for f in os.listdir(".") if f.endswith(".json")]
json_data = [json.load(open(file, "r")) for file in json_files]

# Step 2: Join their content into a pandas dataframe
dataframe = pd.concat([pd.DataFrame(data) for data in json_data], ignore_index=True)

# Ensure that the column names are correct
dataframe.columns = ["model", "cas"]

# Step 3: Group the dataframe by "model" column
grouped_data = dataframe.groupby("model")

# Step 4: For each group, calculate the average value in "cas" column
averages = grouped_data["cas"].mean()

# Step 5: Sort the resulting series
sorted_averages = averages.sort_values()

print(sorted_averages)

ValueError: Length mismatch: Expected axis has 8 elements, new values have 2 elements

### MISTRAL-OPENORCA:7B-Q8_0

In [17]:
import json
import pandas as pd

def load_json_files(directory):
    files = [f for f in sorted(filter(lambda x: x.endswith('.json'), os.listdir(directory))) if x]
    return [json.loads(open(os.path.join(directory, f)).read()) for f in files]

def join_json_content_to_dataframe(json_list):
    df = pd.DataFrame()
    for item in json_list:
        for key in item.keys():
            df[key] = pd.Series([x[key] for x in item])
    return df

def group_by_model(df, column='model'):
    return df.groupby(column).mean()

def sort_resulting_series(series):
    return series.sort_values(ascending=True)

if __name__ == '__main__':
    directory = 'path/to/your/json/files'  # Replace this with the actual path to your JSON files

    json_files = load_json_files(directory)
    df = join_json_content_to_dataframe(json_files)

    model_grouped_df = group_by_model(df, 'model')
    
    result_series = sort_resulting_series(model_grouped_df['cas'])

    print(result_series.head())  # Uncomment to print the top of the sorted series

FileNotFoundError: [WinError 3] Systém nemůže nalézt uvedenou cestu: 'path/to/your/json/files'

In [20]:
import json
import pandas as pd

def load_json_files(directory):
    files = [f for f in sorted(filter(lambda x: x.endswith('.json'), os.listdir(directory))) if x]
    return [json.loads(open(os.path.join(directory, f)).read()) for f in files]

def join_json_content_to_dataframe(json_list):
    df = pd.DataFrame()
    for item in json_list:
        for key in item.keys():
            df[key] = pd.Series([x[key] for x in item])
    return df

def group_by_model(df, column='model'):
    return df.groupby(column).mean()

def sort_resulting_series(series):
    return series.sort_values(ascending=True)

if __name__ == '__main__':
    directory = '/'  # Replace this with the actual path to your JSON files

    json_files = load_json_files(directory)
    df = join_json_content_to_dataframe(json_files)

    model_grouped_df = group_by_model(df, 'model')
    
    result_series = sort_resulting_series(model_grouped_df['cas'])

    print(result_series.head())  # Uncomment to print the top of the sorted series

KeyError: 'model'

### MISTRAL-OPENORCA:LATEST

In [18]:
 # Python script to load all JSON files in the current working directory, join their content into a pandas DataFrame, group by the 'model' column, calculate the average value for each group in the 'cas' column, and sort the resulting Series.
import os
from pathlib import Path
import json
import pandas as pd

# 1. Load all JSON files in the current working directory
def load_json_files():
    # Get the list of all JSON files in the current working directory
    file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".json")]

    # Initialize an empty DataFrame to store the data from the JSON files
    df = pd.DataFrame()

    # Loop through all the JSON files and load their content into the DataFrame
    for file_path in file_paths:
        with open(file_path, "r") as f:
            json_data = json.load(f)
            df = pd.concat([df, pd.DataFrame(json_data)])

    return df

# 2. Join their content into a pandas DataFrame
def join_json_content():
    # Load the JSON files in the current working directory and store them in a DataFrame
    dataframe = load_json_files()

    # Return the DataFrame with joined JSON content
    return dataframe

# 3. Group the DataFrame by the 'model' column
def group_by_model():
    # Join the JSON content into a DataFrame
    dataframe = join_json_content()

    # Group the DataFrame by the 'model' column
    grouped_df = dataframe.groupby('model')

    return grouped_df

# 4. Calculate the average value in the 'cas' column for each group
def calculate_average_cas():
    # Group the DataFrame by the 'model' column
    grouped_df = group_by_model()

    # Calculate the average value in the 'cas' column for each group
    avg_cas_series = grouped_df.mean()['cas']

    return avg_cas_series

# 5. Sort the resulting Series
def sort_resulting_series():
    # Calculate the average value in the 'cas' column for each group
    avg_cas_series = calculate_average_cas()

    # Sort the resulting Series by decreasing values
    sorted_avg_cas_series = avg_cas_series.sort_values(ascending=False)

    return sorted_avg_cas_series

if __name__ == '__main__':
    # Call the sorting function to get the sorted Series
    sorted_avg_cas_series = sort_resulting_series()

    # Print the sorted Series
    print(sorted_avg_cas_series)

FileNotFoundError: [WinError 3] Systém nemůže nalézt uvedenou cestu: 'path/to/your/json/files'

### PHI:LATEST

In [22]:
import pandas as pd
import os
import json


# 1. Load all JSON files in the current working directory
json_files = [f for f in os.listdir() if f.endswith(".json")]
dataframes = []
for file_path in json_files:
    with open(file_path) as f:
        dataframe = pd.read_json(f)
    dataframes.append(dataframe)

# 2. Join all dataframes into one dataframe
full_df = pd.concat(dataframes, ignore_index=True)

# 3. Group the dataframe by "model" column
grouped_df = full_df.groupby("model")

# 4. For each group, calculate the average value in "cas" column
average_values = grouped_df["cas"].mean()

# 5. Sort the resulting series
sorted_values = average_values.sort_index()

print(sorted_values)

model
gemma:latest                    1.435662e+11
llama2:13b                      5.101121e+11
llama2:latest                   8.548862e+10
mistral-openorca:7b-q8_0        1.995293e+11
mistral-openorca:latest         1.738037e+11
mistral:latest                  1.181616e+11
phi:latest                      6.311499e+10
solar:latest                    2.522061e+11
vicuna:latest                   1.180802e+11
wizard-vicuna-uncensored:13b    2.544848e+11
Name: cas, dtype: float64


In [23]:
average_values.sort_values()

model
phi:latest                      6.311499e+10
llama2:latest                   8.548862e+10
vicuna:latest                   1.180802e+11
mistral:latest                  1.181616e+11
gemma:latest                    1.435662e+11
mistral-openorca:latest         1.738037e+11
mistral-openorca:7b-q8_0        1.995293e+11
solar:latest                    2.522061e+11
wizard-vicuna-uncensored:13b    2.544848e+11
llama2:13b                      5.101121e+11
Name: cas, dtype: float64

### SOLAR:LATEST

In [25]:
import pandas as pd
import json
from pathlib import Path

# Step 2: Load all JSON files in current working directory
cwd = str(Path().absolute())
json_files = [file for file in list(Path(cwd).glob('*.json')) if file.is_file()]
data_lists = []

# Step 2 (continuation): Parse the loaded JSON files and add the lists of dictionaries to our master list
for json_file in json_files:
    with open(str(json_file)) as file:
        data_list = json.load(file)
        data_lists.extend(data_list)

# Step 3: Create a pandas DataFrame from the combined list of dictionaries
df = pd.DataFrame(data_lists)

# Step 4 (start): Group by 'model' column
grouped_df = df.groupby('model')

# Step 4 (continuation): For each model group, calculate the average value in 'cas' column
grouped_df_averages = {}
for name, group in grouped_df:
    grouped_df_averages[name] = round(group['cas'].mean(), 2)

# Step 5: Create a series from the dictionary created above and sort it
sorted_series = pd.Series(grouped_df_averages).sort_values()

KeyError: 'model'

### WIZARD-VICUNA-UNCENSORED:13B

In [26]:
import os
import pandas as pd
# Get all JSON files in current working directory
files = []
for filename in os.listdir('.'):
 if filename.endswith('.json'):
 files.append(filename)
# Join JSON files into a single dataframe
df = pd.concat([pd.read_json(f) for f in files], axis=1)
# Group the resulting dataframe by "model" column
grouped = df.groupby('model')
# Calculate the average value in "cas" column for each group
avg_cas = grouped['cas'].mean()
# Sort the resulting series by model and cas columns
sorted_df = avg_cas.reset_index().sort_values(by=['model', 'cas'])
print(sorted_df)

IndentationError: expected an indented block after 'if' statement on line 6 (2177043009.py, line 7)

### VICUNA:LATEST

In [27]:
import json
import os
import pandas as pd

def load_json_files(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.json')]
    return [json.loads(f) for f in files]

def group_by_model_and_calculate_average_cas(dataframes):
    grouped = pd.concat([df.groupby('model').mean()['cas'] for df in dataframes])
    return grouped

if __name__ == '__main__':
    json_files = load_json_files('.')
    dataframes = [pd.DataFrame(json_file) for json_file in json_files]
    average_cas = group_by_model_and_calculate_average_cas(dataframes)
    print(average_cas.sort_values('cas'))


JSONDecodeError: Extra data: line 1 column 2 (char 1)